# Overview

In [1]:
from typing import List, Optional
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import *
import string
import re

In [1]:
import apache_beam as beam 

In [2]:
df_utters = pd.read_parquet('data/venv3-00000-of-00001')
df_utters =df_utters.sort_values('hate_score', ascending = False)
df_utters.head()


,conv_id,timestamp,source,text,hate_score,hate_hits
24,ec12f8e13303cfee07733536855588e61f88123a,1593182152000,client,18-year-old Althea Bernstein — who is black or...,1.112,{'white boy': 2}
31,24d83edae09283e3dcf22ab19df4ba2f69c2dfa7,0,client,"For centuries following the Dorian Invasion, t...",0.667,{'slave': 1}
9,99c5218bc02b270a71db4646255c1daf4bfdf8f7,1602852120000,client,'Worrisome' discovery | Climate change wiped o...,0.667,{'homo': 1}
25,6d59a7e07384968356eeb67b26978f3f5f02cab7,1602071781000,expert,Subtitle: More than 500 years after launching ...,0.667,{'slave': 1}
14,3c3c8fbc3710d908d50fb336afaa045ecf73ed43,1602641036000,client,"“You either turn yourself in, kill yourself or...",0.667,{'kill yourself': 1}


In [3]:
df_hate_words = pd.read_csv('data/raw_other/hateful_ngrams.csv')
df_hate_words.set_index('ngram',drop=True,inplace=True)
dict_hateweights = df_hate_words['prophate'].to_dict()
hate_list = list(df_hate_words.index)

In [4]:
dict_hateweights

{'allah akbar': 0.87,
 'blacks': 0.583,
 'chink': 0.467,
 'chinks': 0.542,
 'dykes': 0.602,
 'faggot': 0.489,
 'faggots': 0.675,
 'fags': 0.543,
 'homo': 0.667,
 'inbred': 0.583,
 'nigger': 0.584,
 'niggers': 0.672,
 'queers': 0.5,
 'raped': 0.717,
 'savages': 0.778,
 'slave': 0.667,
 'spic': 0.75,
 'wetback': 0.667,
 'wetbacks': 0.688,
 'whites': 0.556,
 'a dirty': 0.743,
 'a nigger': 0.509,
 'all niggers': 0.859,
 'all white': 0.75,
 'always fuck': 0.556,
 'ass white': 0.542,
 'be killed': 0.571,
 'beat him': 0.575,
 'biggest faggot': 0.667,
 'blame the': 0.533,
 'butt ugly': 0.5,
 'chink eyed': 0.667,
 'chinks in': 0.556,
 'coon shit': 0.667,
 'dumb monkey': 0.667,
 'dumb nigger': 0.667,
 'fag and': 0.462,
 'fag but': 0.556,
 'faggot a': 0.667,
 'faggot and': 0.556,
 'faggot ass': 0.771,
 'faggot bitch': 0.667,
 'faggot for': 0.533,
 'faggot smh': 0.5,
 'faggot that': 0.583,
 'faggots and': 0.667,
 'faggots like': 0.901,
 'faggots usually': 0.778,
 'faggots who': 0.5,
 'fags are': 0

In [ ]:
def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    #tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    
    tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens
    
def ngrams(tokens,n):
    ngrams = zip(*[tokens[i:] for i in range(n)])
    list_gram = [" ".join(ngram) for ngram in ngrams]
    return list_gram

master = []
tokens = tokenize(text)

for n in range(2,4):
    current = ngrams(tokens,n)
    master += current

master += tokens


hate_score = 0
hate_hits = {}

for gram in master:
    if gram in dict_hateweights.keys():
        if gram in hate_hits:
            hate_hits[gram] += 1 
        else:
            hate_hits[gram] = 1
        hate_score += dict_hateweights[gram]


In [ ]:
print(hate_score)

# Vectorize utterances

## Functions

In [ ]:
# Davidson's custom tokenizer
stemmer = PorterStemmer()
def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    #tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

# Function to gather and tally hateful n-grams found in a given article and calculate the "hatescore" for that article
def tally_counts_doc(row):
    row2 = row[row > 0]
    score = 0
    hits = {}
    for index,val in row2.items():
        hits[index]=val
        hit = dict_hateweights[index] * val
        score += hit 
    row['hate_score'] = score
    row['hate_hits'] = hits

    return row

In [ ]:
df_utters.head()


## Vectorization

In [ ]:
# __________________Load DF of Hateful terms
df_hate_words = pd.read_csv('data/raw_other/hateful_ngrams.csv')
df_hate_words.set_index('ngram',drop=True,inplace=True)
dict_hateweights = df_hate_words['prophate'].to_dict()
hate_list = list(df_hate_words.index)

#____________________ Load text to vectorize

cols = ['conv_id',  'timestamp', 'source', 'text']
df_utters = pd.read_parquet('out/combined_utterances.parquet',names=cols)
#df_utters = df_utters[df_utters['source'] == 'client']

df_utters = df_utters.head(12000) #-------------------------------abbrveiate sample
df_utters.reset_index(inplace=True,drop=True)

#____________________instantiate and fit the vectorizer



unlab_vectors = unlab_cvect.fit(df_utters['text'])
unlab_matrix = unlab_vectors.transform(df_utters['text'])
df_vectors = pd.DataFrame(unlab_matrix.todense(),columns=unlab_vectors.get_feature_names())





## Tally hate score

In [ ]:
#______________________________________Gather hate counts and calculate hate scores

df_vectors = df_vectors.apply(lambda row:tally_counts_doc(row),axis=1,result_type='expand')

df_hatecounts = pd.concat([df_utters,df_vectors],axis=1)

df_hatecounts['hate_hits'] = df_hatecounts['hate_hits'].astype('string')



In [ ]:
df_hatecounts.head()

In [ ]:
df_hatecounts = df_hatecounts[df_hatecounts['hate_score'] > 0]

In [ ]:
df_hate_counts = df_hatecounts.sort_values('hate_score',ascending = False)

In [ ]:
df_hate_counts

In [ ]:
df_counts = df_vectors.sum(axis=0)
df_counts.sort_values(inplace=True,ascending = False)
df_counts = df_counts[df_counts > 0]
df_counts

# Spark vectorization

In [ ]:
import findspark


import pyspark
import random
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType
import pyspark.sql.functions as F
from pyspark.ml.feature import NGram
findspark.init()

sc.stop()
sc = SparkContext('local')
spark = SparkSession(sc)

cols = ['conv_id',  'timestamp', 'source', 'text']
df_utters = spark.read.csv('out/article_data_df_1_1.ftr.csv')
df_utters = df_utters.selectExpr("_c0 as conv_id","_c1 as timestamp","_c2 as source","_c3 as text")

df_utters = df_utters.limit(5000)

#___________________________  Tokenize

stemmer = PorterStemmer()
def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    #tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    tokens = re.split("[^a-zA-Z]*", tweet.lower().strip())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

token_udf = udf(tokenize,ArrayType(StringType()))
#df_utters  = df_utters.withColumn("tokens", token_udf(F.col('text')))

from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
df_utters = tokenizer.transform(df_utters)
#____________________________ n-grams


for i in range(1,5):
    
    ngram = NGram(n=i,inputCol="tokens",outputCol='n' + str(i))

    df_utters = ngram.transform(df_utters)
    
add_lists = F.udf(lambda a,b,c,d : a + b + c + d,ArrayType(StringType()))
df_utters = df_utters.withColumn('n_all',add_lists('n1','n2','n3','n4'))


In [ ]:
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType

col_len = F.udf(lambda a: len(a),IntegerType())

df_utters = df_utters.withColumn('token_len',col_len('tokens'))
df_utters = df_utters.withColumn('n_all_len',col_len('n_all'))

for i in range(2,5):
    df_utters = df_utters.withColumn('n'+ str(i) + '_len',col_len('n'+ str(i)))

In [ ]:
df_utters.where(df_utters['n_all_len'] <= 14).select('n_all_len').show()

In [ ]:
from pyspark.ml.feature import CountVectorizer

# __________________Load DF of Hateful terms
df_hate_words = pd.read_csv('data/raw_other/hateful_ngrams.csv')
df_hate_words.set_index('ngram',drop=True,inplace=True)
dict_hateweights = df_hate_words['prophate'].to_dict()
hate_list = list(df_hate_words.index)

df_utters = df_utters.na.drop("all")

cv = CountVectorizer(inputCol="n_all", outputCol="features")

cvec_model = cv.fit(df_utters)
cvec_model.transform(df_utters).show()


In [ ]:
import dask.dataframe as dd


# Dask?

In [ ]:
cols = ['conv_id',  'timestamp', 'source', 'text']
df_utters = dd.read_csv('out/article_data_df_1_1.ftr.csv',names=cols)

In [ ]:
df_utters

# Local run

In [ ]:
from typing import List, Optional
import pandas as pd
#from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import *
import string
import re
#import apache_beam as beam

import argparse
#from apache_beam.options.pipeline_options import PipelineOptions
from nltk.stem.porter import *
import warnings
warnings.filterwarnings('ignore') 
import pyarrow

stemmer = PorterStemmer()



df_hate_words = pd.read_csv('data/raw_other/hateful_ngrams.csv')
df_hate_words.set_index('ngram',drop=True,inplace=True)
dict_hateweights = df_hate_words['prophate'].to_dict()
hate_list = list(df_hate_words.index)


def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    #tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    
    tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens
    
def ngrams(tokens,n):
    ngrams = zip(*[tokens[i:] for i in range(n)])
    list_gram = [" ".join(ngram) for ngram in ngrams]
    return list_gram



def process(self,element):
    master = []
    tokens = tokenize(element['text'])

    for n in range(2,4):
        current = ngrams(tokens,n)
        master += current
        

    master += tokens

    hate_score = 0
    hate_hits = {}

    for gram in master:
        if gram in dict_hateweights.keys():
            if gram in hate_hits:
                hate_hits[gram] += 1 
            else:
                hate_hits[gram] = 1
            hate_score += dict_hateweights[gram]
    element['hate_score'] = hate_score
    element['hate_hits'] = str(hate_hits)

    return element

df_utters = pd.read_parquet('data/combined_utterances.parquet')

import tqdm
tqdm.pandas()

df_utters_out = df_utters.progress_apply(lambda row:process(row),axis=1,result_type='expand')

